# Poisson Equation

In this example we walk through an exmaple solving the Poisson equation $ -\Delta u(x, y) = f(x, y)$ on a unit square domain. The boundary condition is given by Dirichlet BC $u = \sin(x)$ on all sides.

## Using NGSolve

NGSolve can be imported by 

```
from ngsolve import *
```

The visualization functionality can be imported by 

```
from ngsolve.webgui import Draw
```

In [1]:
# A complete tutorial can be https://docu.ngsolve.org/latest/i-tutorials/

from ngsolve import *
from ngsolve.webgui import Draw

## Mesh

First, we need to define the mesh for the problem. NGSolve provides some default meshing functions (e.g. Circles, Rectangles). 

```
mesh = Mesh(unit_square.GenerateMesh(maxh=0.2)) # it creates a mesh for unit square

```

``maxh`` controls the size of each element.

In [2]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
print('mesh vertices: ', mesh.nv, "\nmesh elements: ", mesh.ne )  # dnumber of vertices & elements

mesh vertices:  134 
mesh elements:  226


In [3]:
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## Refinement of Mesh

If a finer mesh is needed, one can reduce the parameter ``maxh`` or 

```
mesh.Refine()
```

to refine the mesh (of course you can call this procedure multiple times)

In [4]:
mesh.Refine()

Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [5]:
mesh.Refine()

Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## Boundary Condition

To extract boundary information, we can run 

```
mesh.GetBoundaries()
```

In [6]:
mesh.GetBoundaries()

('bottom', 'right', 'top', 'left')

It has 4 boundary pieces labelled by 'bottom', 'right', 'top', 'left'.

## Define A Function

Next, we need to define our $f(x, y)$, say we let $f(x, y) = \sin(x)$. 

In NGSolve, this is done by using coefficient functions, which is very simple. Just

```
f = sin(x)
```

In [7]:
f = sin(x)

## Finite Element Space

The corresponding finite element space is constructed by 

```
fes = H1(mesh, order=2, dirichlet='bottom|right|top|left')
```

``order`` is the polynomial's degree on each element. Higher ``order`` usually gets better results but runs significantly slower. 

The parameter `dirichlet` marks the boundary points outside ``ndof`` (num of degree of freedom). The values on those boundary points must be given by the user. 

In [8]:
fes = H1(mesh, order=2, dirichlet='bottom|right|top|left')

## Weak Formulation

The weak formulation is $\displaystyle\int_D \nabla u\cdot \nabla v_0 dx = -\int_D f v_0 dx$ if $v_0 = 0$ on the Dirichlet BC (or equivalently $H_0^1(D)$). The usual approach splits the solution into 

$$u = u_0 + u_{Dirichlet}$$

where $u_0\in H_0^1(D)$ and $u_{Dirichlet}$ should be understood as the extension of boundary condition into the whole domain.  Thus we actually solves

$$\displaystyle\int_D \nabla u_0\cdot \nabla v_0 dx + \displaystyle\int_D \nabla u_{Dirichlet}\cdot \nabla v_0 dx = -\int_D f v_0 dx$$

or 

$$\displaystyle\int_D \nabla u_0\cdot \nabla v_0 dx = -\int_D f v_0 dx - \displaystyle\int_D \nabla u_{Dirichlet}\cdot \nabla v_0 dx $$


## Extension of BC 

To obtain $u_{Dirichlet}$, we first define the function 

```
g = sin(x)
```

Here ``x`` is a symbol (actually a function) imported from NGSolve.

In [9]:
g = sin(x)

Then use ``GridFunction`` to interpolate the boundary condition into the finite element space. 

In [10]:
u_D = GridFunction(fes)
u_D.Set(g, BND) # BND means interpolation on (near) boundary only

In [11]:
Draw(u_D)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

# Assembly

NGSolve provides an easy way to construct the weak form and assembly. 

```
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
sol = GridFunction(fes)  # solution

```
Sometimes the first two statements are combined into one.
```
u, v = fes.TnT() # Trial and Test
```

The linear and bilinear forms are straightforward

```
a = BilinearForm(grad(u)*grad(v)*dx)
f = LinearForm(0 * v * dx) # since the RHS of Poisson Eq is 0.
```

Finally, assemble them

```

a.Assemble()
f.Assemble()
```


In [12]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
sol = GridFunction(fes)  # solution

In [13]:
a = BilinearForm(grad(u)*grad(v)*dx)
L = LinearForm(f * v * dx) # since the RHS of Poisson Eq is 0.

a.Assemble()
L.Assemble()

## Solve for Free DOFs

In [14]:
r = L.vec - a.mat * u_D.vec

In [15]:
sol.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * r + u_D.vec.data

In [16]:
Draw(sol)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

## Estimate the Error

The exact solution is $u = sin(x)$, we can compare the L2 error squared by 

```
Integrate( (sol - true_sol) **2, mesh)
```

In [19]:
true_sol = sin(x)
Integrate( (sol - true_sol) **2 , mesh)

1.8622375811255602e-14